In [1]:
%pip install 'polars[pandas,pyarrow]'

Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl

In [110]:
df = pl.read_csv("HW1-data.csv")
display(df)

User,"Gender (1 =F, 0=M)",260: Star Wars: Episode IV - A New Hope (1977),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),1: Toy Story (1995),2028: Saving Private Ryan (1998),296: Pulp Fiction (1994),1259: Stand by Me (1986),2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
755,0,1,5,2,null,4,4,2,2,null,3,2,null,5,2,null,4,2,5,null,null
5277,0,5,3,null,2,4,2,1,null,null,4,3,2,2,null,2,null,5,1,3,null
1577,1,null,null,null,5,2,null,4,null,null,1,null,1,4,4,1,1,2,3,1,3
4388,0,null,3,null,null,null,1,2,3,4,null,null,4,1,3,5,null,5,1,1,2
1202,1,4,3,4,1,4,1,null,4,null,1,5,1,null,4,null,3,5,5,null,null
3823,1,2,4,4,4,null,null,3,1,4,4,5,2,4,null,1,null,null,3,null,2
5448,0,null,null,3,1,1,4,null,5,2,null,1,null,null,3,null,1,null,null,5,2
5347,0,4,null,null,null,3,2,2,null,3,null,null,2,3,2,4,null,1,3,5,null
4117,1,5,1,null,4,2,4,4,4,null,1,2,3,1,null,5,null,null,null,null,5


### Assignment 1: Top Movies by Mean Rating

In [38]:
df = pl.read_csv("HW1-data.csv")

df.select(pl.exclude("User").exclude("^Gender.*?$"))\
    .mean()\
    .transpose(include_header=True, header_name="name", column_names=["mean"])\
    .sort("mean", descending=True)\
    .head(3)

name,mean
str,f64
"""318: Shawshank…",3.6
"""260: Star Wars…",3.266667
"""541: Blade Run…",3.222222


### Assignment 1: Top Movies by Mean Count  

In [40]:
df = pl.read_csv("HW1-data.csv")

df.select(pl.exclude("User").exclude("^Gender.*?$"))\
    .count()\
    .transpose(include_header=True, header_name="name", column_names=["count"])\
    .sort("count", descending=True)\
    .head(3)

name,count
str,u32
"""1: Toy Story (…",17
"""593: Silence o…",16
"""260: Star Wars…",15


### Assignment 1: Top Movies by Percent Liking

In [81]:
df = pl.read_csv("HW1-data.csv")

df.select(pl.exclude("User").exclude("^Gender.*?$"))\
    .transpose(include_header=True, header_name="name").melt("name", value_name="rating").group_by("name").agg(
        (pl.when(pl.col("rating") >= 4).then(pl.lit(1)).otherwise(pl.lit(0))).sum().alias("4+"),
        (pl.when(pl.col("rating").is_not_null()).then(pl.lit(1)).otherwise(pl.lit(0))).sum().alias("total")
    ).select(pl.col("name"), (pl.col("4+") / pl.col("total") * 100).alias("4+ %")).sort("4+ %", descending=True).head(3)


name,4+ %
str,f64
"""318: Shawshank…",70.0
"""260: Star Wars…",53.333333
"""3578: Gladiato…",50.0


### Assignment 1: Association with Toy Story

In [116]:
df = pl.read_csv("HW1-data.csv")

toy_story_raters = df.select(pl.col("^.*Toy Story.*$").alias("rate"), pl.col("User")).filter(pl.col("rate").is_not_null())

df.select(pl.exclude("^Gender.*?$"))\
    .melt("User", value_name="rate", variable_name="name")\
    .join(toy_story_raters, on="User", how="left", suffix="_toy_story")\
    .group_by("name").agg(
        pl.when(pl.col("rate_toy_story").is_not_null() & pl.col("rate").is_not_null()).then(pl.lit(1)).otherwise(pl.lit(0)).sum().alias("both"),
        pl.when(pl.col("rate_toy_story").is_not_null()).then(pl.lit(1)).otherwise(pl.lit(0)).sum().alias("toy_story"),
    ).with_columns((pl.col("both") / pl.col("toy_story")).alias("assoc")).sort("assoc", descending=True).head(5)

name,both,toy_story,assoc
str,i32,i32,f64
"""1: Toy Story (…",17,17,1.0
"""260: Star Wars…",14,17,0.823529
"""780: Independe…",13,17,0.764706
"""1210: Star War…",13,17,0.764706
"""593: Silence o…",13,17,0.764706


### Assignment 1: Correlation with Toy Story

In [6]:
df = pl.read_csv("HW1-data.csv")

# The polars corr function uses numpy.corrcoef, which doesn't skip nulls. 
# This difference in results from the CORREL Excel function means that the values don't pass Coursera's checks.
# The corr implementation in pandas and Excel seems the same. So, I need to change the polars frame to pandas, calculate corr, and then change it back again.
corr_matrix_df = df.select(pl.exclude("User").exclude("^Gender.*?$")).to_pandas().corr()

pl.concat([
    df.select(pl.exclude("User").exclude("^Gender.*?$")).transpose(include_header=True, header_name="name").select("name"), 
    pl.from_pandas(corr_matrix_df).select(pl.col("1: Toy Story (1995)").alias("corr_toy_story"))
], how='horizontal').sort("corr_toy_story", descending=True).head(4)

name,corr_toy_story
str,f64
"""1: Toy Story (…",1.0
"""318: Shawshank…",0.888523
"""34: Babe (1995…",0.811107
"""296: Pulp Fict…",0.709842


### Assignment 1: Male-Female Differences in Average Rating

In [71]:
df = pl.read_csv("HW1-data.csv")

mean_stats_df = df.select(pl.exclude("User")).rename({"Gender (1 =F, 0=M)": "gender"}).melt("gender", value_name="rating", variable_name="name").group_by(["gender", "name"]).agg(
    pl.mean("rating").alias("mean")
)

male_mean_stats_df = mean_stats_df.filter(pl.col("gender") == 0).sort("mean", descending=True)
female_mean_stats_df = mean_stats_df.filter(pl.col("gender") == 1).sort("mean", descending=True)

diff_df = male_mean_stats_df.join(female_mean_stats_df, on="name", how="inner").select("name", (pl.col("mean") - pl.col("mean_right")).alias("diff"))

# Male over Female
display(diff_df.sort("diff", descending=True).head(3))

# Female over Male
display(diff_df.select(pl.col("name"), pl.col("diff") * -1).sort("diff", descending=True).head(3))

display(
    df.select(pl.exclude("User")).rename({"Gender (1 =F, 0=M)": "gender"}).melt("gender", value_name="rating", variable_name="name").group_by("gender").agg(
        pl.mean("rating").alias("overall_mean")
    ).sort("gender", descending=True).select(pl.first("overall_mean") - pl.last("overall_mean"))
)


name,diff
str,f64
"""1198: Raiders …",1.666667
"""527: Schindler…",1.0
"""2571: Matrix, …",0.742857


name,diff
str,f64
"""2396: Shakespe…",2.107143
"""34: Babe (1995…",1.428571
"""296: Pulp Fict…",1.375


overall_mean
f64
0.041857


### Assesment 1: Male-Female differences in Liking

In [91]:
df = pl.read_csv("HW1-data.csv")

def count_if(series):
    return sum(x >= 4 for x in series)

stats_df = df.select(pl.exclude("User")).rename({"Gender (1 =F, 0=M)": "gender"})\
    .melt("gender", value_name="rating", variable_name="name")\
    .with_columns((pl.col("rating") >= 4).alias("4+"))\
    .with_columns((pl.col("rating").is_not_null()).alias("not_empty"))\
    .group_by(["name", "gender"]).agg(
        (pl.sum("4+") / pl.sum("not_empty")).alias("4+ %")
    )

male_stats_df = stats_df.filter(pl.col("gender") == 0).sort("4+ %", descending=True)
female_stats_df = stats_df.filter(pl.col("gender") == 1).sort("4+ %", descending=True)

diff_df = male_stats_df.join(female_stats_df, on="name", how="inner").select("name", (pl.col("4+ %") - pl.col("4+ %_right")).alias("diff"))

# Male over Female
display(diff_df.sort("diff", descending=True).head(3))

# Female over Male
display(diff_df.select(pl.col("name"), pl.col("diff") * -1).sort("diff", descending=True).head(3))

name,diff
str,f64
"""1198: Raiders …",0.5
"""2571: Matrix, …",0.371429
"""527: Schindler…",0.333333


name,diff
str,f64
"""2396: Shakespe…",0.75
"""34: Babe (1995…",0.571429
"""541: Blade Run…",0.55
